In [1]:
!pip install plotly

!pip install lightgbm

!pip install pyarrow

# 科学计算模块
import matplotlib
import numpy as np
import pandas as pd

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly
from sklearn.model_selection import train_test_split, KFold
import numpy as np

from sklearn.metrics import accuracy_score
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
train_data_original1 = pd.read_feather('./input/train.feather')
labels1 = pd.read_csv('/openbayes/input/input0/train_labels.csv')




train_data_original=train_data_original1.iloc[0:3616080]
labels=labels1.iloc[0:300000]

cross_validation_test_data=train_data_original1.iloc[3616080:]
cross_validation_test_label=labels1.iloc[300000:]

In [3]:
lgb_config = {
    'lgb_params': {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'dart',
        'max_depth': -1,
        'num_leaves': 64,
        'learning_rate': 0.035,
        'bagging_freq': 5,
        'bagging_fraction': 0.75,
        'feature_fraction': 0.05,
        'min_data_in_leaf': 256,
        'max_bin': 63,
        'min_data_in_bin': 256,
        'tree_learner': 'serial',
        'boost_from_average': 'false',
        'lambda_l1': 0.1,
        'lambda_l2': 30,
        'num_threads': 24,
        'verbosity': 1,
    },
    'rounds': 4500,
    #'early_stopping_rounds': 100,
    'verbose_eval': 50,
}
# 定义早停函数
#early_stopping = lgb.early_stopping(lgb_config['early_stopping_rounds'], verbose=lgb_config['verbose_eval'])


In [4]:
def optimized_preprocessing(df, mode):
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    ignore_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68",
                       'S_2', 'customer_ID', 'target']
    numeric_features = [col for col in df.columns if col not in ignore_features]
    target_features = ['target']

    # print("This is in [2], Columns before processing:"+",".join(df.columns))

    # 使用更高效的方式填充缺失值
    # 使用 .loc[] 方法安全地替换缺失值
    df.loc[:, numeric_features] = df[numeric_features].fillna(df[numeric_features].median())
    df.loc[:, cat_features] = df[cat_features].fillna(df[cat_features].mode().iloc[0])


    # 类别特征的独热编码
    encoder = OneHotEncoder()
    encoded_data = encoder.fit_transform(df[cat_features]).toarray()
    new_feature_names = encoder.get_feature_names_out(cat_features)
    encoded_df = pd.DataFrame(encoded_data, columns=new_feature_names)
    final_df = pd.concat([df.drop(cat_features, axis=1), encoded_df], axis=1)

    # 数值特征处理
    final_df[numeric_features] = np.floor(df[numeric_features] * 100)

    # 时间特征处理
    final_df["S_2"] = pd.to_datetime(df["S_2"])

    # 聚合特征
    if mode == 0:
        agg_funcs = {
            "S_2": ['count'],
            **{name: ['last', 'nunique'] for name in new_feature_names},
            **{name: ['mean', 'std', 'min', 'max'] for name in numeric_features},
            label_name: ['last']  # 添加对标签的聚合操作
        }
    else:
        agg_funcs = {
            "S_2": ['count'],
            **{name: ['last', 'nunique'] for name in new_feature_names},
            **{name: ['mean', 'std', 'min', 'max'] for name in numeric_features},
        }

    final_agg_df = final_df.groupby("customer_ID", sort=False).agg(agg_funcs).reset_index(drop=False)

    new_feature_names = ['_'.join(x) for x in final_agg_df.columns]
    final_agg_df.columns = new_feature_names
    # 如果 'target_last' 存在于列名中，将其重命名为 'target'

    if 'target_last' in final_agg_df.columns:
        final_agg_df = final_agg_df.rename(columns={'target_last': 'target'})

    if 'customer_ID_' in final_agg_df.columns:
        final_agg_df = final_agg_df.rename(columns={'customer_ID_': 'customer_ID'})

    customer_ids = final_agg_df['customer_ID']
    final_agg_df.reset_index(drop=False, inplace=True)
    final_agg_df = final_agg_df.drop(columns=['customer_ID'])

    # print("This is in [2], final_agg_df.columns: "+",".join(final_agg_df.columns))

    return final_agg_df, new_feature_names, customer_ids

In [5]:
def My_best_model_training(train_data, new_feature_names, lgb_config):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    models = []  # 存储每个折的模型

    for train_index, test_index in kf.split(train_data):
        X_train_fold = train_data.iloc[train_index].drop(columns=['target'])
        y_train_fold = train_data.iloc[train_index]['target']
        X_valid_fold = train_data.iloc[test_index].drop(columns=['target'])
        y_valid_fold = train_data.iloc[test_index]['target']

        # 将特征名称更新为新的特征名称
        lgb_config['feature_name'] = new_feature_names

        # 将训练数据和标签转换为LightGBM的Dataset对象
         # Create LightGBM Dataset objects for training and validation
        dtrain = lgb.Dataset(X_train_fold, label=y_train_fold)
        dvalid = lgb.Dataset(X_valid_fold, label=y_valid_fold)

        My_best_model = lgb.train(
            lgb_config['lgb_params'],
            dtrain,
            num_boost_round=lgb_config['rounds'],
            valid_sets=[dvalid],
            #verbose_eval=lgb_config['verbose_eval']
        )


        # 获取验证集上的预测概率
        probs = My_best_model.predict(X_valid_fold)
        pred_labels = (probs >= 0.5).astype(int)

        # 计算精度
        accuracy = accuracy_score(y_valid_fold, pred_labels)
        accuracies.append(accuracy)
        models.append(My_best_model)

    # 返回交叉验证的平均精度和每个折的模型
    return models, np.mean(accuracies)

In [6]:
train_data = pd.merge(train_data_original, labels, on='customer_ID', how='left')
label_name = 'target'

train_data, new_feature_names, _ = optimized_preprocessing(train_data, 0)

# 移除 'customer_ID_'
if 'customer_ID_' in new_feature_names:
    new_feature_names.remove('customer_ID_')
if 'S_2_count' in new_feature_names:
    new_feature_names.remove('S_2_count')

# 将 'target_last' 替换为 'target'
if 'target_last' in new_feature_names:
    index = new_feature_names.index('target_last')
    new_feature_names[index] = 'target'

test_data, useless, customer_ids = optimized_preprocessing(cross_validation_test_data, 1)

In [7]:
lgb_train_data = lgb.Dataset(train_data.drop(columns=['target']), label=train_data["target"])

models,mean_accuracy=My_best_model_training(train_data,new_feature_names,lgb_config)

print("Mean accuracy:",mean_accuracy)

[LightGBM] [Info] Number of positive: 62382, number of negative: 177618
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055302 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28565
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 763
[LightGBM] [Info] Number of positive: 62414, number of negative: 177586
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060285 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28496
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 764
[LightGBM] [Info] Number of positive: 62404, number of negative: 177596
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead o

In [8]:
# 找出训练数据和测试数据中特征的不一致
train_features = set(train_data.columns)
test_features = set(test_data.columns)

# 在训练数据中存在但在测试数据中不存在的特征
missing_in_test = train_features - test_features

# 在测试数据中存在但在训练数据中不存在的特征
missing_in_train = test_features - train_features

# 输出缺失的特征
print("Missing in Test: ", missing_in_test)
print("Missing in Train: ", missing_in_train)

# 处理缺失的特征
# 您可以根据具体情况选择添加缺失的特征或删除多余的特征
# 为测试集中缺失的特征添加默认值
for feature in missing_in_test:
    if feature != 'target':  # 排除 'target'，因为它不应该在测试集中
        
        test_data[feature] = 0

Missing in Test:  {'target'}
Missing in Train:  set()


In [9]:
# 对测试数据进行概率预测
test_probs = [model.predict(test_data) for model in models]
test_prob = np.mean(test_probs, axis=0)
test_class = (test_prob > 0.5).astype(int)


In [10]:
# 创建一个包含 customer_ID 和预测概率的 DataFrame
results_df = pd.DataFrame({
    'customer_ID': customer_ids,
    'prediction': test_class
})

# 将 DataFrame 保存为 CSV 文件
result_csv = results_df.to_csv('predictions_cross_validation_lightGBM.csv', index=False)

In [12]:
print(cross_validation_test_label.shape)
print(test_class.shape)

(158913, 2)
(158913,)
